# Looping with Globs
It can be tedious and error prone to individually specify the files we want to loop over.  Regular expressions, in the form of bash globs allow us to automatically select groups of files to loop over.

## Shell Variables
Assign the variables in this notebook.

In [ ]:
source bioinf_intro_config.sh
mkdir -p $TRIMMED $STAR_OUT

## Globs?
Previously we specified the files we wanted to loop over explicitly:

In [ ]:
for FASTQ in 27_MA_P_S38_L002_R1 27_MA_P_S38_L001_R1
    do
        echo "RUNNING FASTQ: ${FASTQ}"
    done

This is appropriate in some situations, but often when there is a group of files that we want to work with, we can find a simple way to list these files without specifying them one-by-one.  For example, since the reads for each FASTQ are split across four lanes, we might want to analyze the data from all four lanes at once.  There are several ways that we can specify the FASTQs from all four lanes of FASTQ 27_MA:

1. The easiest is to use the `*` wildcard, which matches any number of any characters (including zero), so we can match the FASTQs from all four lanes like this: `$RAW_FASTQS/27_MA*`
2. `*` is easy and useful, but often it is better to be more specific, otherwise we might match something unintentionally. Since the only difference in the names between the lanes is the "L001", "L002", "L003", and "L004", we can narrow our match using `?`, which matches any single character: `27_MA_P_S38_L00?_R1_001.fastq.gz`
3. The best approach is to specify exactly what characters we are allowing in that variable position.  Square brackets allow you to list specific characters that can match `[1234]` or a range `[1-4]`: `27_MA_P_S38_L00[1-4]_R1_001.fastq.gz`

Globs can be confusing.  Keep in mind that here we are using the globs to search through all the file names in a directory and list the files with a name that matches a specific pattern.

In [ ]:
echo $RAW_FASTQS/27_MA*

In [ ]:
echo $RAW_FASTQS/27_MA_P_S38_L00?_R1_001.fastq.gz

In [ ]:
echo $RAW_FASTQS/27_MA_P_S38_L00[1234]_R1_001.fastq.gz

In [ ]:
echo $RAW_FASTQS/27_MA_P_S38_L00[1-4]_R1_001.fastq.gz

## More Complex Globs

We can combine multiple wildcards in a glob to match a more complex set of filenames.  For example we could match MA samples 10 through 19, Lanes 1 through 4, with the following glob.

In [ ]:
ls $RAW_FASTQS/1?_MA_*_L00[1-4]_R1_001.fastq.gz

## Looping over a glob
Now we can put together `for` loops with globs, to loop over all the lanes from library 27_MA

In [ ]:
for FASTQ in $RAW_FASTQS/27_MA_P_S38_L00[1-4]_R1_001.fastq.gz
    do
        echo "RUNNING FASTQ: ${FASTQ}"
    done

## String Manipulation

But we still have a bit of work to do.  Before when we were manually specifying each FASTQ, we looped over a *substring* of the full path, then added on the prefix and suffix, for example: `$TRIMMED/${FASTQ}_001.trim.fastq.gz`.  But now we are grabbing the full path, and we need to manipulate it so that we can generate output file names that are different than the input, and put the output in different directories.  We can do all this with the `basename` bash function. The simple form of `basename` removes the whole directory portion of a path and just returns the filename:

In [ ]:
for FASTQ in $RAW_FASTQS/27_MA_P_S38_L00[1-2]_R1_001.fastq.gz
    do
        echo "FULL PATH IS: ${FASTQ}"
        echo "basename gives us: $(basename ${FASTQ})"
        echo ""
    done

If you give `basename` a string as a second argument, it will strip that string from the end of the path (if it is there):

In [ ]:
for FASTQ in $RAW_FASTQS/27_MA_P_S38_L00[1-2]_R1_001.fastq.gz
    do
        echo "FULL PATH IS: ${FASTQ}"
        echo "basename gives us: $(basename ${FASTQ} '_001.fastq.gz')"
        echo ""
    done

> Note that if the string you give is not a suffix of the path, nothing is stripped from the *end* of the path, but the directory prefix will still be removed:

In [ ]:
for FASTQ in $RAW_FASTQS/27_MA_P_S38_L00[1-2]_R1_001.fastq.gz
    do
        echo "FULL PATH IS: ${FASTQ}"
        echo "with '_001.fastq.gz': $(basename ${FASTQ} '_001.fastq.gz')"
        echo "with 'fastq':         $(basename ${FASTQ} 'fastq')"
        echo "with 'foo':           $(basename ${FASTQ} 'foo')"
        echo ""
    done

And we can assign the results of `basename` to a variable for later use:

In [ ]:
for FASTQ in $RAW_FASTQS/27_MA_P_S38_L00[1-2]_R1_001.fastq.gz
    do
        echo "FULL PATH IS: ${FASTQ}"
        FASTQ_BASE="$(basename ${FASTQ} '_001.fastq.gz')"
        echo "basename gives us: $FASTQ_BASE"
        echo "OUTPUT PATH: ${TRIMMED}/${FASTQ_BASE}_001.trim.fastq.gz"
    done

With globs and `basename` in our toolbox, we are ready to **conquer the world** or at least run multiple FASTQs through our pipeline, without breaking a sweat!

## A globy pipeline

In [ ]:
for FASTQ in $RAW_FASTQS/27_MA_P_S38_L00[1-4]_R1_001.fastq.gz
    do
        FASTQ_BASE="$(basename ${FASTQ} '_001.fastq.gz')"
        echo "---------------- TRIMMING: $FASTQ_BASE ----------------"
        fastq-mcf \
            $MYINFO/neb_e7600_adapters.fasta \
            $RAW_FASTQS/${FASTQ_BASE}_001.fastq.gz \
            -q 20 -x 0.5 \
            -o $TRIMMED/${FASTQ_BASE}_001.trim.fastq.gz
        
        echo "---------------- MAPPING: $FASTQ_BASE ----------------"
        STAR \
            --runMode alignReads \
            --twopassMode None \
            --genomeDir $GENOME_DIR \
            --readFilesIn $TRIMMED/${FASTQ_BASE}_001.trim.fastq.gz \
            --readFilesCommand gunzip -c \
            --outFileNamePrefix ${STAR_OUT}/${FASTQ_BASE}_ \
            --quantMode GeneCounts
    done

### And let's check the result

In [ ]:
ls ${STAR_OUT}

In [ ]:
head ${STAR_OUT}/27_MA_P_S38_L00?_R1_ReadsPerGene.out.tab

## Glob References
- [Globbing Section](http://tldp.org/LDP/abs/html/globbingref.html) from [The Advanced Bash-Scripting Guide](http://tldp.org/LDP/abs/html/index.html)
- [Glob](https://en.wikipedia.org/wiki/Glob_%28programming%29) article on Wikipedia